In [24]:
import dataflows as DF

In [25]:
URL = 'https://docs.google.com/spreadsheets/d/1gFV5jreypDMAzXmerIaqBoxX7tOXYkb7wu0UA3wcf5M/edit?usp=sharing'

In [26]:
TYPES = {
    'מחרוזת': 'string',
    'מספר שלם': 'integer',
    'מספר עשרוני': 'number',
    'תאריך': 'date',
    'כן/לא': 'boolean',
    'קישור בודד': 'string',
    'מזעץ': 'string',
    'טקסט חופשי': 'string',
    'מחרוזת מספרית': 'string',
    'מחרוזות מופרדות בפסיק': 'string',
    'שנה/תאריך': 'date',
    'רשימת קישורים מופרדים בפסיק': 'string',
}

def get_options(row):
    if row['טיפוס הנתונים'] == 'קישור בודד':
        return {
            'format': 'uri'
        }
    elif 'בפסיק' in row['טיפוס הנתונים']:
        return {
            'commaSeparated': True
        }
    return dict()


ct = DF.Flow(
    DF.load(URL, headers=3),
    DF.checkpoint('taxonomy'),
    DF.add_field('name', 'string', lambda r: r['מזהה']),
    DF.add_field('title', 'string', lambda r: r['שם השדה']),
    DF.add_field('description', 'string', lambda r: r['הסבר']),
    DF.add_field('dataType', 'string', lambda r: TYPES[r['טיפוס הנתונים']]),
    DF.add_field('mandatory', 'boolean', lambda r: r['חובה'] == 'כן'),
    DF.add_field('options', 'object', get_options),
    DF.select_fields(['name', 'title', 'description', 'dataType', 'mandatory', 'options']),
    DF.printer(tablefmt='html'),
).results()[0][0]

using checkpoint data from .checkpoints/taxonomy


#,name (string),title (string),description (string),dataType (string),mandatory (boolean),options (object)
1,meta:date,תאריך עדכון המידע,"למשל, תאריך ביצוע הסקר",date,True,{}
2,meta:source,מזהה מקור המידע,"למשל, ״סקר עצים ירוחם״ או ״רשות המים״",string,True,{}
3,meta:url,קישור למקור המידע,קישור למקור ממנו התקבל המידע,string,True,{'format': 'uri'}
4,meta:source-type,סוג מקור המידע,תיאור של הארגון ממנו הגיע המידע,string,True,{}
5,meta:collection-type,אופן איסוף המידע,תיאור של האופן בו נאסף המידע,string,True,{}
6,meta:tree-id,מזעץ,המזהה הארצי של העץ,string,False,{}
7,meta:internal-id,מזהה העץ,"מזהה פנימי, תלוי מקור, של העץ",string,False,{}
8,location:x,קו אורך,"קואורדינטת X, בהתאם לרשת שנבחרה",number,True,{}
9,location:y,קו רוחב,"קואורדינטת Y, בהתאם לרשת שנבחרה",number,True,{}
10,location:grid,רשת קואורדינטות,הרשת ממנה נגזרו הקואורדינטות,string,False,{}


In [27]:
import json

with open('column_types.json', 'w') as out:
    json.dump(ct, out, indent=2, ensure_ascii=False)